# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

We begin with some helper functions

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Dict, Optional, Iterable, Set, Union, Tuple
import hashlib
from jsonpath_ng import parse as jsonpath_parse
import json
import re

_HEX_RE = re.compile(r'^[0-9a-fA-F]*$')

def hex_to_bytes(h: str) -> bytes:
    """
    Convert a hex string to raw bytes.

    Accepts optional '0x' prefix and common separators (space, underscore,
    hyphen, colon). Raises TypeError for non-str inputs and ValueError for
    invalid hex or odd length after cleaning.
    """
    if not isinstance(h, str):
        raise TypeError("hex_to_bytes() expects a str")

    s = h.strip()
    if s.startswith(("0x", "0X")):
        s = s[2:]

    # Remove common separators
    for sep in (" ", "_", "-", ":"):
        s = s.replace(sep, "")

    if len(s) % 2 != 0:
        raise ValueError(f"hex string has odd length ({len(s)}): {h!r}")

    if not _HEX_RE.fullmatch(s):
        raise ValueError(f"invalid hex string: {h!r}")

    return bytes.fromhex(s)

def sha256_hex(data: bytes) -> str:
    """Get the hex digest of the SHA-256 hash of the given data"""
    return hashlib.sha256(data).hexdigest()

def path_for_key(key: str) -> str:
    """Get a JSONPath for a given key using bracket notation."""
    safe = key.replace('"', r'\"')
    return f'$["{safe}"]'

def extend_path(base: str, key_or_index: Union[str, int]) -> str:
    """Append a segment to a JSONPath using bracket notation."""
    if isinstance(key_or_index, str):
        seg = path_for_key(key_or_index)[1:]  # '["key"]'
        return f"{base}{seg}"
    if isinstance(key_or_index, int):
        return f"{base}[{key_or_index}]"
    raise TypeError("Path extension must be str (field) or int (index).")

def hash_json(data: Any) -> bytes:
    """Compute a SHA-256 hash of JSON-serializable data."""
    j = json.dumps(data, separators=(',', ':'), sort_keys=True)
    return hashlib.sha256(j.encode('utf-8')).digest()

In [ ]:
#| export

@dataclass(frozen=True)
class OperationHeader:
    """Metadata about an operation."""
    kind: str # e.g. 'observe' or 'transform' 
    task: str # e.g. 'fetch_http', 'parse_article'
    tool: str # e.g. 'requests', 'my_scraper_1'
    output_type: str # e.g. 'http_response', 'article'
    event_uuid: Optional[str] # e.g. '550e8400-e29b-41d4-a716-446655440000'
    timestamp: Optional[str] # e.g. '2024-01-01T12:00:00Z'

    def sha256(self) -> bytes:
        """
        Deterministic SHA-256 over all fields as plain data.
        Uses a canonical JSON *list* to preserve field order.
        """
        return hash_json([
            self.kind, self.task, self.tool, self.output_type, self.event_uuid, 
            self.timestamp
        ])

In [ ]:
#| export

# -------- JSON leaf tags --------
OP_TAG  = "$op"          # {"$op": {"id": "<hex>", "path": "$.a.b[0]"}}
ART_TAG = "$artifact"    # {"$artifact": "<sha256-hex>"}

class Artifact:
    """Lazy-loaded bytes referenced by SHA-256; caches after first load."""
    __slots__ = ("book", "sha256", "_cache")
    def __init__(self, book: "Scrapebook", sha256: bytes):
        self.book = book
        self.sha256 = sha256
        self._cache: Optional[bytes] = None
    def bytes(self) -> bytes:
        if self._cache is None:
            self._cache = self.book._fetch_artifact_bytes_sha256(self.sha256)
        return self._cache
    def value(self) -> bytes:
        return self.bytes()
    def __repr__(self) -> str:
        return f"<Artifact {self.sha256.hex()}>"


class Operation:
    """Lazy header; decoded results/inputs are cached along with their dependency sets."""
    __slots__ = (
        "book", "id", "_header",
        "_results_decoded_cache",
        "_result_artifact_deps_cache",
        "_inputs_decoded_cache",
        "_input_artifact_deps_cache", "_input_op_deps_cache",
    )
    def __init__(self, book: "Scrapebook", op_id: bytes):
        self.book = book
        self.id = op_id
        self._header: Optional[OperationHeader] = None

        # Results caches
        self._results_decoded_cache: Optional[Any] = None
        self._result_artifact_deps_cache: Optional[Set[str]] = None

        # Inputs caches
        self._inputs_decoded_cache: Optional[Any] = None
        self._input_artifact_deps_cache: Optional[Set[str]] = None
        self._input_op_deps_cache: Optional[Set[bytes]] = None

    # --- header (lazy) ---
    @property
    def header(self) -> OperationHeader:
        if self._header is None:
            self._header = self.book._fetch_operation_header(self.id)
        return self._header

    @property
    def kind(self) -> str: return self.header.kind
    @property
    def task(self) -> str: return self.header.task
    @property
    def tool(self) -> str: return self.header.tool
    @property
    def output_type(self) -> str: return self.header.output_type
    @property
    def event_uuid(self) -> Optional[str]: return self.header.event_uuid
    @property
    def timestamp(self) -> Optional[str]: return self.header.timestamp

    # --- result-path handles ---
    def __getitem__(self, key: str) -> "OperationResult":
        return self.at(path_for_key(key))
    def at(self, path: str) -> "OperationResult":
        return OperationResult(self, path)

    # --- results JSON (raw / decoded + deps) ---
    def results_json(self) -> Any:
        return self.book._fetch_results_json(self.id)

    def _ensure_results_decoded(self) -> Any:
        if self._results_decoded_cache is None:
            raw = self.book._fetch_results_json(self.id)
            arts = set()
            decoded = self.book.decode_json(raw, arts=arts)
            self._results_decoded_cache = decoded
            self._result_artifact_deps_cache = arts
        return self._results_decoded_cache

    def results_decoded(self) -> Any:
        return self._ensure_results_decoded()

    def result_artifact_deps(self) -> Set[str]:
        if self._result_artifact_deps_cache is None:
            self._ensure_results_decoded()
        return self._result_artifact_deps_cache or set()

    # --- inputs JSON (raw / decoded + deps) ---
    def inputs_json(self) -> Any:
        return self.book._fetch_inputs_json(self.id)

    def _ensure_inputs_decoded(self) -> Any:
        if self._inputs_decoded_cache is None:
            raw = self.book._fetch_inputs_json(self.id)
            arts = set()
            ops = set()
            decoded = self.book.decode_json(raw, ops=ops, arts=arts)
            self._inputs_decoded_cache = decoded
            self._input_artifact_deps_cache = arts
            self._input_op_deps_cache = ops
        return self._inputs_decoded_cache

    def inputs_decoded(self) -> Any:
        return self._ensure_inputs_decoded()

    def input_artifact_deps(self) -> Set[str]:
        if self._input_artifact_deps_cache is None:
            self._ensure_inputs_decoded()
        return self._input_artifact_deps_cache or set()

    def input_op_deps(self) -> Set[bytes]:
        if self._input_op_deps_cache is None:
            self._ensure_inputs_decoded()
        return self._input_op_deps_cache or set()

    # --- JSON leaf builders for constructing inputs ---
    def op_ref(self) -> Dict[str, Any]:
        return {OP_TAG: {"id": self.id.hex(), "path": "$"}}
    
    def out(self, path: str) -> Dict[str, Any]:
        return {OP_TAG: {"id": self.id.hex(), "path": path}}

    # --- values via decoded cache ---
    def value(self) -> Any:
        return self.results_decoded()

    def __repr__(self) -> str:
        return f"<Operation {self.id.hex()}>"


@dataclass(frozen=True)
class OperationResult:
    """
    Handle: (Operation, JSONPath) into that operation's decoded results.
    Supports subscripting to extend the path: op["a"]["b"][0]
    """
    op: Operation
    path: str

    def __getitem__(self, key_or_index: Union[str, int]) -> "OperationResult":
        """Extend the JSONPath with a field or index."""
        return OperationResult(self.op, extend_path(self.path, key_or_index))

    def value(self) -> Any:
        """
        Evaluate JSONPath on the op's **decoded cached** results.
        - 0 matches -> None
        - 1 match   -> the value
        - >1 matches-> list of values
        """
        decoded = self.op._ensure_results_decoded()
        expr = jsonpath_parse(self.path)
        matches = [m.value for m in expr.find(decoded)]
        if not matches:
            return None
        if len(matches) == 1:
            return matches[0]
        return matches

    # JSON leaf for niceness (like op.op_ref/out)
    def op_ref(self) -> Dict[str, Any]:
        """Return the JSON leaf representation of this result reference."""
        return {OP_TAG: {"id": self.op.id.hex(), "path": self.path}}

    def __repr__(self) -> str:
        return f"<OperationResult id={self.op.id.hex()} path={self.path!r}>"


# -------- Recorder (fixed config) --------
class Recorder:
    """
    Fixed (kind, task, tool, output_type).
    """

    __slots__ = ("book", "kind", "task", "tool", "output_type")

    def __init__(self, book: "Scrapebook", *, kind: str, task: str, tool: str, output_type: str):
        self.book = book
        self.kind = kind
        self.task = task
        self.tool = tool
        self.output_type = output_type

    def record(self, inputs: Any, results: Any) -> Operation:
        op_deps: Set[bytes] = set()
        input_arts: Set[bytes] = set()
        output_arts: Set[bytes] = set()
        # allow op refs; collect deps
        enc_inputs  = self._encode(inputs, ops=op_deps, arts=input_arts) 
        # forbid op refs in results 
        enc_results = self._encode(results, ops=None, arts=output_arts)    
        op_id = self.book._persist_operation(
            kind=self.kind,
            task=self.task,
            tool=self.tool,
            output_type=self.output_type,
            inputs_json=enc_inputs,
            results_json=enc_results,
            op_deps=op_deps,
            input_arts=input_arts,
            output_arts=output_arts,
            event_uuid=None,
            timestamp=None,
        )
        return Operation(self.book, op_id)

    def _encode(self, obj: Any, *, 
        ops: Optional[Set[bytes]], arts: Optional[Set[bytes]]
    ) -> Any:
        # op refs
        if isinstance(obj, OperationResult):
            if ops is None:
                raise ValueError("Operation references are not allowed here (ops=None).")
            ops.add(obj.op.id)
            return obj.op_ref()
        if isinstance(obj, Operation):
            if ops is None:
                raise ValueError("Operation references are not allowed here (ops=None).")
            ops.add(obj.id)
            return obj.op_ref()
        # artifacts
        if isinstance(obj, Artifact):
            if arts is None:
                raise ValueError("Artifact references are not allowed here (arts=None).")
            arts.add(obj.sha256)
            return {ART_TAG: obj.sha256.hex()}
        if isinstance(obj, (bytes, bytearray, memoryview)):
            h = self.book._put_artifact_sha256(bytes(obj)).hex()  # -> "<hex>"
            return {ART_TAG: h}

        # containers / primitives
        if isinstance(obj, dict):
            return {k: self._encode(v, ops=ops, arts=arts) for k, v in obj.items()}
        if isinstance(obj, (list, tuple)):
            return [self._encode(v, ops=ops, arts=arts) for v in obj]
        return obj


# -------- Scrapebook (DB façade; stubs + ALWAYS-collect decoder) --------
class Scrapebook:
    def __init__(self):
        # map operation ids to headers + inputs + outputs
        self.ops = dict()
        self.arts = dict()
        pass

    def recorder(self, *, kind: str, task: str, tool: str, output_type: str) -> Recorder:
        return Recorder(self, kind=kind, task=task, tool=tool, output_type=output_type)

    def obs_recorder(self, *, task: str, tool: str, output_type: str) -> Recorder:
        return self.recorder(kind="observe", task=task, tool=tool, output_type=output_type)

    def trans_recorder(self, *, task: str, tool: str, output_type: str) -> Recorder:
        return self.recorder(kind="transform", task=task, tool=tool, output_type=output_type)

    # persistence
    def _persist_operation(
        self,
        *,
        kind: str,
        task: str,
        tool: str,
        output_type: str,
        inputs_json: Any,
        results_json: Any,
        op_deps: Set[bytes],
        input_arts: Set[bytes],
        output_arts: Set[bytes],
        event_uuid: Optional[str],
        timestamp: Optional[str],
    ) -> bytes:
        #TODO: store op deps
        header = OperationHeader(kind, task, tool, output_type, event_uuid, timestamp)
        op_id = hash_json([header.sha256().hex(), inputs_json, results_json])
        self.ops[op_id] = { "header" : header, "input" : inputs_json, "result" : results_json}
        return op_id

    # header
    def _fetch_operation_header(self, op_id: bytes) -> OperationHeader:
        return self.ops[op_id]["header"]

    # JSON fetch
    def _fetch_results_json(self, op_id: bytes) -> Any:
        return self.ops[op_id]["result"]
    
    def _fetch_inputs_json(self, op_id: bytes) -> Any:
        return self.ops[op_id]["input"]

    # artifacts (fixed SHA-256)
    def _put_artifact_sha256(self, data: bytes) -> bytes:
        """Store bytes content-addressed by SHA-256 and return the hex hash."""
        hash = hashlib.sha256(data).digest()
        self.arts[hash] = data
        return hash
    
    def _fetch_artifact_bytes_sha256(self, sha256: bytes) -> bytes:
        return self.arts[sha256]

    def decode_json(
            self, node: Any, *, ops: Set[bytes] = None, arts: Set[bytes] = None) -> Any:
        """
        Decode JSON leaves back to handles, collecting dependencies along the way.

        - Ops decode to lazy OperationResult and their IDs are collected
          If `ops` is None, operation references are forbidden and will raise ValueError.
        - Artifacts decode to lazy Artifact and their hashes are collected
          If `arts` is None, artifact references are forbidden and will raise ValueError.

        Returns: decoded_tree
        """
        if isinstance(node, dict):
            if OP_TAG in node:
                if ops is None:
                    raise ValueError("Operation references are not allowed here (ops=None).")
                try:
                    spec = node[OP_TAG]
                    op = Operation(self, hex_to_bytes(spec["id"]))
                    path = spec.get("path", "$")
                except:
                    raise ValueError(f"Invalid operation reference: {node!r}")
                ops.add(op.id)
                return OperationResult(op, path)
            if ART_TAG in node:
                if arts is None:
                    raise ValueError("Artifact references are not allowed here (arts=None).")
                try:
                    sha = hex_to_bytes(node[ART_TAG])
                except:
                    raise ValueError(f"Invalid artifact reference: {node!r}") 
                arts.add(sha)
                return Artifact(self, sha)
            return {k: self.decode_json(v, ops=ops, arts=arts) for k, v in node.items()}
        if isinstance(node, list):
            return [self.decode_json(v, ops=ops, arts=arts) for v in node]
        return node

In [ ]:
book = Scrapebook()

In [ ]:
scrapes = book.obs_recorder(task="scrape_site", tool="demo", output_type="http_response")

In [ ]:
op = scrapes.record({"url": "http://example.com"}, {"status": 200, "content": b"hello"})

In [ ]:
op

<Operation 0cea232512a0908ab4eb409dccfe03f566ee9f36f2c6dda01ab97910cd21ed5b>

In [ ]:
(op.kind, op.task, op.tool, op.output_type)

('observe', 'scrape_site', 'demo', 'http_response')

In [ ]:
op.results_json()

{'status': 200,
 'content': {'$artifact': '2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'}}

In [ ]:
op.results_decoded()

{'status': 200,
 'content': <Artifact 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824>}

In [ ]:
op["content"].value().bytes()

b'hello'

In [ ]:
op["content"]

<OperationResult id=0cea232512a0908ab4eb409dccfe03f566ee9f36f2c6dda01ab97910cd21ed5b path='$["content"]'>

In [ ]:
articles = book.trans_recorder(task="parse_article", tool="demo2", output_type="article")

In [ ]:
op2 = articles.record({"content" : op["content"], "mode": "cool"}, "good article")

In [ ]:
op2

<Operation 9251d9e6006363cedcb4bd14c98f80d4a4de22e5b0af763cf087ee75f73e14ca>

In [ ]:
op2.value()

'good article'

In [ ]:
op2.inputs_decoded()

{'content': <OperationResult id=0cea232512a0908ab4eb409dccfe03f566ee9f36f2c6dda01ab97910cd21ed5b path='$["content"]'>,
 'mode': 'cool'}

In [ ]:
[dep.hex() for dep in op2.input_op_deps()]

['0cea232512a0908ab4eb409dccfe03f566ee9f36f2c6dda01ab97910cd21ed5b']

In [ ]:
op3 = articles.record(
    {"content" : op["content"], "mode": op2, "status": op["status"]}, 
    "better article"
)

In [ ]:
op3

<Operation fb667362593df912db68321709893d297fab97d78f3ba13ff795bc6fdf6abeba>

In [ ]:
op3.value()

'better article'

In [ ]:
[dep.hex() for dep in op3.input_op_deps()]

['9251d9e6006363cedcb4bd14c98f80d4a4de22e5b0af763cf087ee75f73e14ca',
 '0cea232512a0908ab4eb409dccfe03f566ee9f36f2c6dda01ab97910cd21ed5b']

In [ ]:
op

<Operation 0cea232512a0908ab4eb409dccfe03f566ee9f36f2c6dda01ab97910cd21ed5b>

In [ ]:
op2

<Operation 9251d9e6006363cedcb4bd14c98f80d4a4de22e5b0af763cf087ee75f73e14ca>

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()